In [7]:
# !/usr/bin/python
# -*- coding: utf-8
from __future__ import division, unicode_literals

%matplotlib inline
# http://matplotlib.org/users/pyplot_tutorial.html
import matplotlib.pyplot as plt
import numpy as np
from pandas import Series, DataFrame
import pandas as pd

# http://nbviewer.ipython.org/
import xlsxwriter
from pandas import ExcelWriter
import time
import datetime
from datetime import timedelta
import operator
import re



In [8]:
file_name ="data/user_raw.xlsx"
xls = pd.ExcelFile(file_name)
dt_raw = xls.parse('sheet1')

dt_raw= dt_raw.reindex(dt_raw.index.drop(0))
#dt_act.index = dt_act['활동명']
#del dt_act['활동명']

In [9]:
list_benefit_all = [u'(\uc9c0\uc6d0 \ud61c\ud0dd) \ud65c\ub3d9\ube44 \uc9c0\uae09',
 u'(\uc9c0\uc6d0 \ud61c\ud0dd) \uc790\uc0ac\uc81c\ud488 \ubb34\ub8cc \uc9c0\uae09 (ex: \ubdf0\ud2f0 \uae30\uc5c5 \ud654\uc7a5\ud488 \uc0d8\ud50c \uc9c0\uae09/\uc758\ub958 \uae30\uc5c5 \uc637 \ubb34\ub8cc \uc9c0\uae09 \ub4f1)',
 u'(\uad50\uc721 \ud61c\ud0dd) \uc2e4\ubb34 \uad50\uc721 \uc81c\uacf5',
 u'(\uad50\uc721 \ud61c\ud0dd) \uc804\ubb38\uac00/\uc2e4\ubb34\uc790 \uba58\ud1a0\ub9c1',
 u'(\uacbd\ud5d8 \ud61c\ud0dd) \ud589\uc0ac \ucc38\uc5ec \uae30\ud68c',
 u'(\uacbd\ud5d8 \ud61c\ud0dd) \ud65c\ub3d9\uc790\uac04\uc758 \uad50\ub958 \uae30\ud68c \uc81c\uacf5(\ub2e8\ud569\ub300\ud68c/\uc6cc\ud06c\uc0f5/MT \ub4f1)',
 u'(\uc2a4\ud399 \ud61c\ud0dd) \uc218\ub8cc\uc99d \ubc0f \uc778\uc99d\uc11c \uc81c\uacf5',
 u'(\uc2a4\ud399 \ud61c\ud0dd) \uc785\uc0ac\uc2dc \uac00\uc0b0\uc810 \uc81c\uacf5/\uc778\ud134\uc2ed \uae30\ud68c \uc81c\uacf5 \uc81c\uacf5',
 u'(\uc2a4\ud399 \ud61c\ud0dd) \uc784\uc9c1\uc6d0 \ub9cc\ub0a8 \uae30\ud68c \uc81c\uacf5']

list_interest_all =['문화/역사', '여행/호텔/항공','정치/사회', '교육',
                    '콘텐츠/엔터테인먼트', '디자인/사진/예술/영상', '뷰티/미용/화장품', '환경/에너지',
                    '과학/공학/기술/IT', '의료/헬스/체육', '미디어/커뮤니케이션', '경영/컨설팅/경제/금융',
                    '요리/음식', '창업', '자기계발(독서/발표/취미)']

list_industry_all =['서비스업', '', '제조/화학', '', '의료/제약/복지', '', '유통/무역/운송', '','교육업','',
                    '건설업', '','IT/웹/통신','', '미디어/디자인', '','은행/금융', '','기관/협회','']

In [10]:
"""
    b1 활동비
    b2 자사제품
    b3 실무교육
    b4 전문가/실무자 멘토링
    b5 행사참여
    b6 활동자간 교류
    b7 수료증 및 인증서
    b8 입사시 가산점
    b9 임직원 만남
"""

def user_by_persona(persona):
    dt_result = DataFrame()
    dt_result = dt_result.append([["email", "persona", "start", "end", "region", "category", 
                               "i1","i2","i3","i4","i5","i6","i7","i8","i9","i10","i11","i12","i13","i14","i15","i16",
                               "b1", "b2", "b3",  "b4", "b5", "b6", "b7", "b8", "b9",
                               "organization", "d1","d2","d3","d4","d5","d6","d7","d8","d9","d10"]])
    for i in range (1, dt_raw.index.size+1):
        if(dt_raw['persona'][i] in persona):
            result_code = [dt_raw['email'][i], dt_raw['persona'][i]]


            if(type(dt_raw['time_start'][i])==float): result_code.append(datetime.date(2000,1,1))
            else : result_code.append(dt_raw['time_start'][i])
            
            if(type(dt_raw['time_end'][i])==float): result_code.append(datetime.date(2100,12,31))
            else : result_code.append(dt_raw['time_end'][i])
                
                
            result_code.append(dt_raw['region'][i])
            result_code.append(dt_raw['category'][i])
            list_benefit = []
            list_interest =[]
            list_industries = [] 

            if(type(dt_raw['interests'][i])==float): 
                list_interest = list_interest_all
            else: 
                list_interest = dt_raw['interests'][i].split(", ")
            for j in range(0, 16):
                if(j<len(list_interest)):result_code.append(list_interest[j])
                else : result_code.append("-")

            if(type(dt_raw['benefit'][i])==float): list_benefit=list_benefit_all
            else: list_benefit = dt_raw['benefit'][i].split(", ")
            for a in list_benefit_all :
                if (a in list_benefit) : 
                    result_code.append("O")
                else : result_code.append("X")


            result_code.append(dt_raw['companies'][i])

            if(type(dt_raw['industries'][i])==float): 
                list_industries = list_industry_all
            else: 
                list_industries = re.split("\), | \(", dt_raw['industries'][i])
            for j in range(0, 20):
                if(j<len(list_industries)) : 
                    if(j%2==0): result_code.append(list_industries[j])
                else : 
                    if(j%2==0): result_code.append("-")

            dt_result = dt_result.append([result_code])
            
    return dt_result

In [11]:
list_persona = []
list_etc = []
for i in range(1, dt_raw.index.size+1):
    if not ( dt_raw['persona'][i] in list_persona):
        list_persona.append(dt_raw['persona'][i])
for l in list_persona:
    print l
    list_etc.append(l)
    
list_etc.remove("스펙과 무관한 다양한 경험, 새로운 사람들 만나보기.")
list_etc.remove("전공과 관련한 실무 경험과 교육, 역량 쌓기.")
list_etc.remove("취업에 도움되는 스펙 쌓기.")
list_etc

전공과 관련한 실무 경험과 교육, 역량 쌓기.
스펙과 무관한 다양한 경험, 새로운 사람들 만나보기.
취업에 도움되는 스펙 쌓기.


[]

In [6]:
writer = ExcelWriter('data/user_polished.xlsx')
all_per = user_by_persona(list_persona)
per1 = user_by_persona(["스펙과 무관한 다양한 경험, 새로운 사람들 만나보기."])
per2 = user_by_persona(["전공과 관련한 실무 경험과 교육, 역량 쌓기."])
per3 = user_by_persona(["취업에 도움되는 스펙 쌓기."])
etc = user_by_persona(list_etc)

all_per.to_excel(writer, sheet_name='All', engine='xlsxwriter')
per1.to_excel(writer, sheet_name='Per1', engine='xlsxwriter')
per2.to_excel(writer, sheet_name='Per2', engine='xlsxwriter')
per3.to_excel(writer, sheet_name='Per3', engine='xlsxwriter')
etc.to_excel(writer, sheet_name='Etc', engine='xlsxwriter')
writer.save()

In [14]:
list_category=dt_raw['category'][i].split(", ")
for c in list_category:
    print c

서포터즈
해외탐방
동아리
멘토링
강연


In [15]:
li = [1,2,3]
for l in li :
    print l

1
2
3
